In [1]:
import yfinance as yf
import pandas as pd
stock_names=['BHARTIARTL.NS',"SBIN.NS","DRREDDY.NS","GSPL.NS","JSWSTEEL.NS","JSWENERGY.BO","RELIANCE.NS","YESBANK.NS","TATAMOTORS.NS","LAURUSLABS.NS","NTPC.NS","SAIL.BO"]
stocks=None
for stock in range(len(stock_names)):
    df=yf.download(stock_names[stock], start="2015-01-01", end="2023-01-01") 
    df["stock"]=stock_names[stock]
    df=df.drop("Adj Close",axis=1)
    if stock!=0:
        stocks=pd.concat([stocks,df])
    else:
        stocks=df
stocks=stocks.sort_index()        
stocks=stocks.reset_index(drop=False) 

amount=100000
profit=0 
dates=sorted(set(stocks["Date"])) 
# print(len(dates))
stocks[:12]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume,stock
0,2015-01-01,325.402405,335.043274,322.877411,333.206940,2414849,BHARTIARTL.NS
1,2015-01-01,114.449997,114.900002,113.250000,113.599998,200199,GSPL.NS
2,2015-01-01,104.889999,107.199997,104.199997,106.589996,2674400,JSWSTEEL.NS
3,2015-01-01,102.699997,103.800003,102.000000,102.500000,456255,JSWENERGY.BO
4,2015-01-01,312.450012,315.000000,310.700012,314.000000,6138488,SBIN.NS
5,2015-01-01,438.988831,441.812073,437.676270,439.781311,1367720,RELIANCE.NS
6,2015-01-01,153.800003,154.929993,153.210007,154.509995,3675025,YESBANK.NS
7,2015-01-01,82.650002,83.400002,82.000000,82.650002,1094040,SAIL.BO
8,2015-01-01,489.252289,494.149750,487.026154,493.160370,944551,TATAMOTORS.NS
9,2015-01-01,120.166664,120.208336,118.416664,118.708336,1478980,NTPC.NS


In [6]:
selected_stocks=[]
stock_history=pd.DataFrame({"Date":[],"Name of the stocks":[],"Transaction Price":[],"Event(Sell/Buy)":[],"Total Units":[]}) 
total_units=10
amount=100000

every_stock_history={}
for value in stock_names:
    every_stock_history[value]={"previous_price":0,"no_of_stocks":0,"previous_total_amount":0}    


def buy_at_opening(stock_name,date,opening_price,closing_price,volume,stocks_to_sold_evening):
    history_row={"Date":[],"Name of the stocks":[],"Transaction Price":[],"Event(Sell/Buy)":[],"Total Units":[]}
    no_of_stocks=3500
    global amount
    global stock_history 
    if volume<no_of_stocks:
        no_of_stocks=volume
    
    amount_to_spend=no_of_stocks*opening_price 
    
    amount_we_get=(no_of_stocks * closing_price)*0.95
    if amount_to_spend < amount and amount_we_get > amount_to_spend:
        print(amount,end=" ")
        amount-=amount_to_spend 
        print(stock_name,opening_price,amount_to_spend,no_of_stocks,date,sep="---BUY--")
        print("--------------------------------------------------------")
        history_row["Date"].append(date)
        history_row["Name of the stocks"].append(stock_name)
        history_row["Transaction Price"].append("Open") 
        history_row["Event(Sell/Buy)"].append("BUY")
        history_row["Total Units"].append(no_of_stocks)  
        
        every_stock_history[stock_name]["previous_price"]=opening_price 
        every_stock_history[stock_name]["no_of_stocks"]=no_of_stocks 
        every_stock_history[stock_name]["previous_total_amount"]=amount_to_spend
        stock_history=pd.concat([stock_history,pd.DataFrame(history_row)]) 
        stocks_to_sold_evening.append(stock_name) 
    elif amount_to_spend > amount:
        max_stocks_we_can_buy = amount // opening_price 
        if max_stocks_we_can_buy<=0:
            return 
        amount_to_spend = max_stocks_we_can_buy * opening_price 
        amount_we_get= (max_stocks_we_can_buy* closing_price)*0.95 
        if amount_we_get > amount_to_spend:
            print(amount,"No Stocks",end=" ")
            amount-=amount_to_spend 
            
            print(stock_name,opening_price,closing_price,amount_to_spend,max_stocks_we_can_buy,date,sep="---BUY--")
            print("--------------------------------------------------------")
            history_row["Date"].append(date)
            history_row["Name of the stocks"].append(stock_name)
            history_row["Transaction Price"].append("Open") 
            history_row["Event(Sell/Buy)"].append("BUY")
            history_row["Total Units"].append(max_stocks_we_can_buy)  

            every_stock_history[stock_name]["previous_price"]=opening_price 
            every_stock_history[stock_name]["no_of_stocks"]=max_stocks_we_can_buy 
            every_stock_history[stock_name]["previous_total_amount"]=amount_to_spend
            stock_history=pd.concat([stock_history,pd.DataFrame(history_row)]) 
            stocks_to_sold_evening.append(stock_name) 
            
    
            
def sell_at_evening(stock_name,date,opening_price,closing_price,volume,stocks_to_sold_evening): 
    global amount 
    global stock_history 
    history_row={"Date":[],"Name of the stocks":[],"Transaction Price":[],"Event(Sell/Buy)":[],"Total Units":[]}
    current_stock_history=every_stock_history[stock_name] 
    sold_price=(current_stock_history["no_of_stocks"]*closing_price)*0.95
#     print(amount,end=" ")
    amount+=sold_price
    history_row["Date"].append(date)
    history_row["Name of the stocks"].append(stock_name)
    history_row["Transaction Price"].append("Close") 
    history_row["Event(Sell/Buy)"].append("SELL")
    history_row["Total Units"].append(current_stock_history["no_of_stocks"]) 
    stock_history=pd.concat([stock_history,pd.DataFrame(history_row)]) 
    stocks_to_sold_evening.remove(stock_name)
    every_stock_history[stock_name]["previous_price"]=0
    every_stock_history[stock_name]["no_of_stocks"]=0
    every_stock_history[stock_name]["previous_total_amount"]=0

for value in stock_names:
    every_stock_history[value]={"previous_price":0,"no_of_stocks":0}
# print(every_stock_history)   
template={"Date":[],"Name of the stocks":[],"Transaction Price":[],"Event(Sell/Buy)":[],"Total Units":[]}
for date in dates:
    stocks_on_date=stocks[stocks["Date"]==date]
    stocks_on_date=stocks_on_date.sort_values("Close",ascending=False)
#     print(stocks_on_date)
    stocks_to_sold_evening=[]
    stocks_left_with_out_buying_at_opening=[]
    for stock in range(len(stocks_on_date)):
        stock=stocks_on_date.iloc[stock] 
        opening_price=stock["Open"]
        closing_price=stock["Close"]
        volume=stock["Volume"]
        stock_name=stock["stock"]
        if opening_price<closing_price:
            buy_at_opening(stock_name,date,opening_price,closing_price,volume,stocks_to_sold_evening)
        elif closing_price<opening_price:
            stocks_left_with_out_buying_at_opening.append(stock_name)
#             sell_at_opening(stock_name,date,opening_price,closing_price,volume,stocks_to_sold_tommorow) 
            
    for stock in range(len(stocks_on_date)):
        stock=stocks_on_date.iloc[stock] 
        opening_price=stock["Open"]
        closing_price=stock["Close"]
        volume=stock["Volume"]
        stock_name=stock["stock"]
        if stock_name in stocks_to_sold_evening:
            sell_at_evening(stock_name,date,opening_price,closing_price,volume,stocks_to_sold_evening)


        
        

print(amount-100000)
# print(stocks)
stock_history



100000 No Stocks GSPL.NS---BUY--112.69999694824219---BUY--122.75---BUY--99964.89729309082---BUY--887.0---BUY--2015-01-05 00:00:00
--------------------------------------------------------
103470.39020690917 No Stocks SBIN.NS---BUY--288.0---BUY--306.95001220703125---BUY--103392.0---BUY--359.0---BUY--2015-02-13 00:00:00
--------------------------------------------------------
104763.69187011717 No Stocks JSWSTEEL.NS---BUY--97.87999725341797---BUY--103.76000213623047---BUY--104731.59706115723---BUY--1070.0---BUY--2015-02-19 00:00:00
--------------------------------------------------------
105504.13698043821 No Stocks BHARTIARTL.NS---BUY--314.0628967285156---BUY--337.8437194824219---BUY--105211.07040405273---BUY--335.0---BUY--2015-03-10 00:00:00
--------------------------------------------------------
107811.83030166624 No Stocks BHARTIARTL.NS---BUY--339.7259826660156---BUY--357.6763916015625---BUY--107693.13650512695---BUY--317.0---BUY--2015-03-11 00:00:00
---------------------------------

182370.99867706283 No Stocks TATAMOTORS.NS---BUY--556.3499755859375---BUY--588.7000122070312---BUY--181926.44201660156---BUY--327.0---BUY--2016-09-06 00:00:00
--------------------------------------------------------
183324.21545257553 SAIL.BO---BUY--48.75---BUY--170625.0---BUY--3500---BUY--2016-09-07 00:00:00
--------------------------------------------------------
185266.7205261229 No Stocks DRREDDY.NS---BUY--3001.0---BUY--3282.550048828125---BUY--183061.0---BUY--61.0---BUY--2016-11-09 00:00:00
--------------------------------------------------------
2205.7205261228955 No Stocks TATAMOTORS.NS---BUY--501.1499938964844---BUY--533.5---BUY--2004.5999755859375---BUY--4.0---BUY--2016-11-09 00:00:00
--------------------------------------------------------
201.12055053695804 No Stocks JSWSTEEL.NS---BUY--155.00999450683594---BUY--168.0850067138672---BUY--155.00999450683594---BUY--1.0---BUY--2016-11-09 00:00:00
--------------------------------------------------------
192456.86664199812 No Stock

353565.7009037013 No Stocks BHARTIARTL.NS---BUY--329.62603759765625---BUY--351.891845703125---BUY--353359.1123046875---BUY--1072.0---BUY--2018-07-30 00:00:00
--------------------------------------------------------
358573.2442630763 No Stocks TATAMOTORS.NS---BUY--250.0---BUY--265.04998779296875---BUY--358500.0---BUY--1434.0---BUY--2018-08-01 00:00:00
--------------------------------------------------------
361150.8426334376 No Stocks JSWSTEEL.NS---BUY--336.04998779296875---BUY--354.79998779296875---BUY--360917.68688964844---BUY--1074.0---BUY--2018-08-24 00:00:00
--------------------------------------------------------
362235.58328895515 No Stocks JSWSTEEL.NS---BUY--371.3999938964844---BUY--399.0---BUY--362114.99404907227---BUY--975.0---BUY--2018-08-29 00:00:00
--------------------------------------------------------
369694.3392398829 No Stocks GSPL.NS---BUY--184.5---BUY--195.39999389648438---BUY--369553.5---BUY--2003.0---BUY--2018-09-06 00:00:00
----------------------------------------

725194.3153533926 SAIL.BO---BUY--30.450000762939453---BUY--106575.00267028809---BUY--3500---BUY--2019-09-04 00:00:00
--------------------------------------------------------
725850.5626831045 TATAMOTORS.NS---BUY--109.75---BUY--384125.0---BUY--3500---BUY--2019-09-05 00:00:00
--------------------------------------------------------
734241.8228301993 TATAMOTORS.NS---BUY--127.0---BUY--444500.0---BUY--3500---BUY--2019-09-11 00:00:00
--------------------------------------------------------
289741.82283019926 YESBANK.NS---BUY--66.0---BUY--231000.0---BUY--3500---BUY--2019-09-11 00:00:00
--------------------------------------------------------
743525.5880508414 No Stocks RELIANCE.NS---BUY--1176.795166015625---BUY--1242.5716552734375---BUY--742557.7497558594---BUY--631.0---BUY--2019-09-20 00:00:00
--------------------------------------------------------
967.8382949819788 No Stocks SBIN.NS---BUY--274.5---BUY--301.70001220703125---BUY--823.5---BUY--3.0---BUY--2019-09-20 00:00:00
------------------

1329917.2434816349 SAIL.BO---BUY--21.0---BUY--73500.0---BUY--3500---BUY--2020-03-31 00:00:00
--------------------------------------------------------
1333058.4909448612 GSPL.NS---BUY--172.4499969482422---BUY--603574.9893188477---BUY--3500---BUY--2020-04-01 00:00:00
--------------------------------------------------------
729483.5016260135 YESBANK.NS---BUY--22.0---BUY--77000.0---BUY--3500---BUY--2020-04-01 00:00:00
--------------------------------------------------------
1341423.5041627872 No Stocks DRREDDY.NS---BUY--3250.0---BUY--3583.199951171875---BUY--1339000.0---BUY--412.0---BUY--2020-04-07 00:00:00
--------------------------------------------------------
2423.504162787227 No Stocks RELIANCE.NS---BUY--1091.7513427734375---BUY--1194.7747802734375---BUY--2183.502685546875---BUY--2.0---BUY--2020-04-07 00:00:00
--------------------------------------------------------
240.00147724035196 No Stocks JSWSTEEL.NS---BUY--148.0500030517578---BUY--158.10000610351562---BUY--148.0500030517578---B

1676355.7422082887 LAURUSLABS.NS---BUY--206.8000030517578---BUY--723800.0106811523---BUY--3500---BUY--2020-08-10 00:00:00
--------------------------------------------------------
1700647.499791907 JSWENERGY.BO---BUY--51.25---BUY--179375.0---BUY--3500---BUY--2020-08-13 00:00:00
--------------------------------------------------------
1700822.499791907 NTPC.NS---BUY--89.4000015258789---BUY--312900.0053405762---BUY--3500---BUY--2020-08-17 00:00:00
--------------------------------------------------------
1704795.0045984255 YESBANK.NS---BUY--14.100000381469727---BUY--49350.00133514404---BUY--3500---BUY--2020-08-18 00:00:00
--------------------------------------------------------
1705486.2538974748 NTPC.NS---BUY--94.5999984741211---BUY--331099.9946594238---BUY--3500---BUY--2020-08-20 00:00:00
--------------------------------------------------------
1710543.7541645037 LAURUSLABS.NS---BUY--215.55999755859375---BUY--754459.9914550781---BUY--3500---BUY--2020-08-21 00:00:00
----------------------

2296733.3502681716 SAIL.BO---BUY--94.8499984741211---BUY--331974.9946594238---BUY--3500---BUY--2021-04-26 00:00:00
--------------------------------------------------------
2298920.855608748 No Stocks JSWSTEEL.NS---BUY--672.0---BUY--726.5---BUY--2298912.0---BUY--3421.0---BUY--2021-04-29 00:00:00
--------------------------------------------------------
2361097.5306087476 SAIL.BO---BUY--111.19999694824219---BUY--389199.98931884766---BUY--3500---BUY--2021-04-30 00:00:00
--------------------------------------------------------
2369068.7811428052 BHARTIARTL.NS---BUY--530.0---BUY--1855000.0---BUY--3500---BUY--2021-05-03 00:00:00
--------------------------------------------------------
514068.78114280524 SAIL.BO---BUY--119.44999694824219---BUY--418074.98931884766---BUY--3500---BUY--2021-05-03 00:00:00
--------------------------------------------------------
95993.79182395758 YESBANK.NS---BUY--12.850000381469727---BUY--44975.00133514404---BUY--3500---BUY--2021-05-03 00:00:00
-------------------

3077076.843012617 JSWENERGY.BO---BUY--197.0---BUY--689500.0---BUY--3500---BUY--2022-06-21 00:00:00
--------------------------------------------------------
3088985.5828655222 JSWENERGY.BO---BUY--212.1999969482422---BUY--742699.9893188477---BUY--3500---BUY--2022-07-11 00:00:00
--------------------------------------------------------
3100561.863840864 LAURUSLABS.NS---BUY--485.95001220703125---BUY--1700825.0427246094---BUY--3500---BUY--2022-07-27 00:00:00
--------------------------------------------------------
1399736.8211162547 YESBANK.NS---BUY--13.75---BUY--48125.0---BUY--3500---BUY--2022-07-27 00:00:00
--------------------------------------------------------
3108874.2798936823 TATAMOTORS.NS---BUY--451.95001220703125---BUY--1581825.0427246094---BUY--3500---BUY--2022-08-01 00:00:00
--------------------------------------------------------
3120389.277757452 JSWENERGY.BO---BUY--247.9499969482422---BUY--867824.9893188477---BUY--3500---BUY--2022-08-02 00:00:00
-------------------------------

,Date,Name of the stocks,Transaction Price,Event(Sell/Buy),Total Units
0,2015-01-05,GSPL.NS,Open,BUY,887.0
0,2015-01-05,GSPL.NS,Close,SELL,887.0
0,2015-02-13,SBIN.NS,Open,BUY,359.0
0,2015-02-13,SBIN.NS,Close,SELL,359.0
0,2015-02-19,JSWSTEEL.NS,Open,BUY,1070.0
...,...,...,...,...,...
0,2022-12-13,YESBANK.NS,Close,SELL,3500.0
0,2022-12-26,JSWENERGY.BO,Open,BUY,3500.0
0,2022-12-26,YESBANK.NS,Open,BUY,3500.0
0,2022-12-26,JSWENERGY.BO,Close,SELL,3500.0


In [7]:
df_transactions = pd.DataFrame(stock_history, columns=["Date", "Name of the stocks", "Transaction Price", "Event(Sell/Buy)", "Total Units"])
df_transactions.to_csv("Coding_Challange.csv", index=False)